<a href="https://colab.research.google.com/github/NSALHI1/Animal-Recognition/blob/main/Example_for_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Animals Recognition**

In [2]:
!pip install transformers
!pip install scipy
!pip install torch
!pip install gradio
!pip install sentencepiece
!pip install timm
!pip install inflect
!pip install phonemizer
!pip install py-espeak-ng
!pip install soundfile
!sudo apt-get update
!sudo apt-get install espeak-ng

from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration
import gradio as gr
import torch
import scipy.io.wavfile as wavfile

# Load translation model to translate English text to Arabic
translator = pipeline("translation_en_to_ar", model="Helsinki-NLP/opus-mt-en-ar")

# Load the BLIP image captioning model and processor
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load a question-answering model for retrieving information
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Load a text-to-speech model for generating audio from text
narrator = pipeline("text-to-speech", model="kakao-enterprise/vits-ljs")

# Combined context for known animals
combined_context = """
Tigers are the largest species among the Felidae and classified in the genus Panthera.
Elephants are the largest land animals on Earth. They are known for their large ears, tusks made of ivory, and their trunks.
Deer are herbivorous mammals forming the family Cervidae. Species include white-tailed deer, mule deer, elk, moose, and reindeer.
Lions are large carnivorous mammals known for their majestic manes and live in social groups called prides.
Penguins are flightless birds primarily found in the Southern Hemisphere, known for their distinctive black and white plumage.
Dogs are domesticated mammals known for their loyalty and intelligence.
Domestic cats are small mammals valued for their companionship and playfulness.
Giraffes are the tallest land animals, known for their long necks.
Zebras are known for their distinctive black and white stripes and live in herds.
Horse are domesticated mammals valued for their strength, speed, and companionship.
Crocodiles are large reptiles found in tropical regions, known for their powerful jaws and stealthy hunting techniques.
Pandas are large bears native to China, distinguished by their black and white coloration.
"""

# List of known animal names for extraction
animal_names = ["tiger", "elephant", "deer", "lion", "penguin", "dog", "cat", "giraffe", "zebra", "dolphin", "panda", "crocodile","Horse"]

# Function to extract animal name from the generated caption
def extract_animal_from_caption(caption):
    for animal in animal_names:
        if animal in caption.lower():
            return animal
    return None

# Function to generate audio from text using the text-to-speech model
def generate_audio(text):
    # Generate the narrated text
    narrated_text = narrator(text)
    # Save the audio to a WAV file
    wavfile.write("output.wav", rate=narrated_text["sampling_rate"], data=narrated_text["audio"][0])
    return "output.wav"

# Function to recognize the animal in the image and provide relevant information
def recognize_animal_and_get_info(image):
    # Step 1: Generate a caption for the uploaded image using BLIP
    inputs = blip_processor(images=image, return_tensors="pt")
    caption_ids = blip_model.generate(**inputs)
    caption = blip_processor.decode(caption_ids[0], skip_special_tokens=True)

    # Step 2: Extract the animal name from the generated caption
    animal_name = extract_animal_from_caption(caption)

    # Step 3: Use the QA model to retrieve information based on the combined context
    if animal_name:
        question = f"Describe a {animal_name}?"
        answer = qa_model(question=question, context=combined_context)
        info = answer['answer']
    else:
        info = "Sorry, I couldn't identify the animal in the image."

    # Translate both the caption and the information to Arabic
    translated_caption = translator(caption)[0]['translation_text']
    translated_info = translator(info)[0]['translation_text']

    # Generate audio from the caption
    audio_file = generate_audio(caption)

    return caption, info, audio_file, translated_caption, translated_info  # Return all outputs

# Define the Gradio interface with tabs for displaying results
with gr.Blocks(css=".gradio-container { background-color: beige; }") as iface:
    gr.Markdown("# Animal Recognition")
    gr.Markdown("Upload an animal image to generate a caption and insights about the identified animal.")

    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload Image")

    with gr.Tab("Generated Results"):
        with gr.Row():
            output_caption = gr.Textbox(label="Caption", interactive=False)
            output_info = gr.Textbox(label="Animal Insight", interactive=False)
            output_audio = gr.Audio(label="Audio", interactive=False)

    with gr.Tab("Translations"):
        translated_caption_output = gr.Textbox(label="Translated Caption", interactive=False)
        translated_info_output = gr.Textbox(label="Translated Animal Insight", interactive=False)

    # Define the action to take when an image is uploaded
    image_input.change(
        fn=recognize_animal_and_get_info,
        inputs=image_input,
        outputs=[output_caption, output_info, output_audio, translated_caption_output, translated_info_output]
    )


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at kakao-enterprise/vits-ljs were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_laye

In [3]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0fce2e0fe6106cfee2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
